In [1]:
#Questionable quality
#https://www.youtube.com/watch?v=nTt_ajul8NY
#https://towardsdatascience.com/understanding-encoder-decoder-sequence-to-sequence-model-679e04af4346

#https://towardsdatascience.com/language-translation-with-rnns-d84d43b40571
#http://www.wildml.com/2016/01/attention-and-memory-in-deep-learning-and-nlp/

#Great quality
#https://machinelearningmastery.com/lstm-autoencoders/
#https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html
#https://keras.io/examples/lstm_seq2seq/
#https://www.youtube.com/watch?v=QuELiw8tbx8&feature=youtu.be&list=PL3FW7Lu3i5Jsnh1rnUwq_TcylNr7EkRe6&t=1190
#https://stackoverflow.com/questions/50915634/multilayer-seq2seq-model-with-lstm-in-keras

In [2]:
import os
import pretty_midi
import numpy as np
import math
import random
import matplotlib.pyplot as plt

In [3]:
from __future__ import print_function

import keras

from keras.models import Model
from keras.models import load_model

from keras.layers import Input, LSTM, Dense

Using TensorFlow backend.


In [4]:
import argparse
import librosa
import sys

In [5]:
workdir = "C:\\Users\\toend\\Documents\\ITU\\Thesis"
pathToSave = os.path.join(workdir, "MIDIdata\\RESULTS")

frequency = 100
START_OF_SEQUENCE_VALUE = 0.10
END_OF_SEQUENCE_VALUE = 0.99

In [6]:
sys.path.append(workdir + '\\Code\\Utils')
from Utils import getDataSets
from Utils import compressDataSet
from Utils import reshape
from Utils import matchOneToAnyNumberOfMatches
from Utils import equalityCheck
from Utils import piano_roll_to_pretty_midi

In [7]:
X1, X2, X3, X4 = getDataSets()

In [8]:
print(X1.shape)
print(X2.shape)
print(X3.shape)
print(X4.shape)

(178, 3000, 128)
(208, 3000, 128)
(128, 3000, 128)
(78, 3000, 128)


In [9]:
X1 = compressDataSet(X1)
X2 = compressDataSet(X2)

In [10]:
Z = matchOneToAnyNumberOfMatches(X2, X1, 10)

In [11]:
random.shuffle(Z)

X, Y = zip(*Z)

X = np.array(X)
Y = np.array(Y)

print(X.shape)
print(Y.shape)

(2080, 1500, 128)
(2080, 1500, 128)


In [12]:
#encoderInputRolls = np.array(X1[:150])
#decoderInputRolls = np.array(X2[:150])
#decoderTargetRolls = np.array(X2[:150])
#---------------------------------------
encoderRolls = X
decoderRolls = np.empty((Y.shape[0], Y.shape[1] + 2, Y.shape[2]))

In [13]:
startToken = np.full((128), 0.01)
endToken = np.full((1, 128), 0.99)

for i in range(len(Y)):
    paddedRoll = np.insert(Y[i], 0, startToken, axis = 0)
    decoderRolls[i] = np.append(paddedRoll, endToken, axis = 0)

In [14]:
print(encoderRolls.shape)
print(decoderRolls.shape)

[0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01]
[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.5  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.85 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0. 

In [10]:
batch_size = 65  # Batch size for training.
epochs = 150  # Number of epochs to train for.
latent_dim = 100  # Latent dimensionality of the encoding space.
pianoNodes = 128
learning_rate = 0.02  #No clue, default is 0.01
decay = learning_rate/epochs  #https://www.pyimagesearch.com/2019/07/22/keras-learning-rate-schedules-and-decay/

In [11]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, pianoNodes))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# Discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [12]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, pianoNodes))
# Set up our decoder to return full output sequences,
# and to return internal states as well. Don't use the
# return states in the training model, but use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
#CONSIDER ACTIVATION FUNCTION --> SIGMOID MIGHT BE BETTER FOR NOW
decoder_dense = Dense(pianoNodes, activation='sigmoid')
decoder_outputs = decoder_dense(decoder_outputs)

In [18]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#Define optimizer
optimizer = keras.optimizers.adam(learning_rate=learning_rate)

# Run training
model.compile(optimizer=optimizer, loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit([encoderInputRolls, decoderInputRolls], decoderTargetRolls,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.1,
          shuffle=True)
# Save model
model.save('LSTMs2sb65l02comp1500shuffleMatchTenEqualPadded.h5')

Train on 1872 samples, validate on 208 samples
Epoch 1/150
1872/1872 [==============================] - 307s 164ms/step - loss: 0.1259 - accuracy: 0.9547 - val_loss: 0.0719 - val_accuracy: 0.9736
Epoch 2/150
1872/1872 [==============================] - 362s 193ms/step - loss: 0.0725 - accuracy: 0.9734 - val_loss: 0.0695 - val_accuracy: 0.9736
Epoch 3/150
1872/1872 [==============================] - 419s 224ms/step - loss: 0.0697 - accuracy: 0.9734 - val_loss: 0.0660 - val_accuracy: 0.9736
Epoch 4/150
1872/1872 [==============================] - 375s 200ms/step - loss: 0.0658 - accuracy: 0.9734 - val_loss: 0.0622 - val_accuracy: 0.9736
Epoch 5/150
1872/1872 [==============================] - 337s 180ms/step - loss: 0.0620 - accuracy: 0.9734 - val_loss: 0.0584 - val_accuracy: 0.9735
Epoch 6/150
1872/1872 [==============================] - 328s 175ms/step - loss: 0.0586 - accuracy: 0.9734 - val_loss: 0.0556 - val_accuracy: 0.9735
Epoch 7/150
1872/1872 [==============================] - 33

1872/1872 [==============================] - 452s 241ms/step - loss: 0.0226 - accuracy: 0.9727 - val_loss: 0.0219 - val_accuracy: 0.9730
Epoch 110/150
1872/1872 [==============================] - 452s 241ms/step - loss: 0.0225 - accuracy: 0.9727 - val_loss: 0.0218 - val_accuracy: 0.9730
Epoch 111/150
1872/1872 [==============================] - 450s 240ms/step - loss: 0.0224 - accuracy: 0.9727 - val_loss: 0.0218 - val_accuracy: 0.9730
Epoch 112/150
1872/1872 [==============================] - 449s 240ms/step - loss: 0.0224 - accuracy: 0.9727 - val_loss: 0.0218 - val_accuracy: 0.9730
Epoch 113/150
1872/1872 [==============================] - 452s 241ms/step - loss: 0.0223 - accuracy: 0.9727 - val_loss: 0.0218 - val_accuracy: 0.9730
Epoch 114/150
1872/1872 [==============================] - 456s 244ms/step - loss: 0.0223 - accuracy: 0.9727 - val_loss: 0.0217 - val_accuracy: 0.9731
Epoch 115/150
1872/1872 [==============================] - 448s 239ms/step - loss: 0.0223 - accuracy: 0.9727

In [13]:
# load model
model = load_model('LSTMs2sb65l02comp1500shuffleMatchTenEqualPadded.h5')
# summarize model.
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 128)    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 128)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 100), (None, 91600       input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 100),  91600       input_2[0][0]                    
                                                                 lstm_1[0][1]               

In [14]:
# Next: inference mode (sampling).
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)



In [15]:
# Decode sequences back to something readable.
# Pick only values above 0.50?

In [64]:
encoderInputRolls = X2

In [65]:
#Decide cutoff value for node-on/off --- 0.09?
def convertProbabilitiesToActualPianoRoll(decodedPianoRoll):
    #print(decodedPianoRoll[0])
    pianoRoll = np.where(decodedPianoRoll<0.98, decodedPianoRoll, 0.90)
    #Everything above 17%?
    pianoRoll = np.where(pianoRoll>0.51, pianoRoll, 0)
    #print(pianoRoll[0])
    
    #pianoRoll = np.where(pianoRoll==0, pianoRoll, 80)
    #print(pianoRoll[0])
    #print(pianoRoll.shape)
    return pianoRoll

In [66]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    # Populate the first character of target sequence with the start character.
    #start = np.array([START_OF_SEQUENCE_VALUE for i in range(128)])
    
    start = np.zeros(128)
    target_seq = np.reshape(start, (1, 1, 128))
    
    #Normalizing the START_OF_SEQUENCE_VALUE from 101 to 1.01
    #target_seq = target_seq/100

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_pianoRoll = np.empty((1,1,128))
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a single node
        #sampled_node = np.argmax(output_tokens[0, -1, :])
        
        #Add all nodes
        addedPianoRoll = np.reshape(output_tokens[0, -1, :], (1,1,128))
        decoded_pianoRoll = np.append(decoded_pianoRoll, addedPianoRoll , axis=0)

        # Exit condition: either hit max length
        # or find stop character ---> sampled_node == 102
        if (len(decoded_pianoRoll) > 1500):
            stop_condition = True

        # Update the target sequence (of length 1).
        #target_seq = np.zeros((1, 1, 128))
        #target_seq[0, 0, sampled_token_index] = 1.
        
        target_seq = np.reshape(output_tokens[0, -1, :], (1,1,128))
        
        #Readjusting values to avoid vanishing values
        #target_seq = target_seq*2
        target_seq = convertProbabilitiesToActualPianoRoll(target_seq)
        
        # Update states
        states_value = [h, c]

    return decoded_pianoRoll

In [67]:
pianoRolls = np.empty((1, 3000, 128))
for seq_index in range(3):
    # Take one sequence (part of the training set) for trying out decoding.
    print('----------------')
    input_seq = encoderInputRolls[seq_index: seq_index + 1]
    print(input_seq.shape) # (1, 3002, 128)
    decoded_pianoRoll = decode_sequence(input_seq)
    print(decoded_pianoRoll.shape) # (3001, 1 , 128) --> can be changed 
    
    #Reshaping to get rid of 3rd dimension
    decoded_pianoRoll = np.reshape(decoded_pianoRoll, (decoded_pianoRoll.shape[0], 128))
    print(decoded_pianoRoll.shape)
    
    #deleting start of sequence
    decoded_pianoRoll = np.delete(decoded_pianoRoll, 0, 0)
    #print(decoded_pianoRoll.shape)
    
    #converting to actual pianoRoll with only chosen node-on/off
    #pianoRoll = decoded_pianoRoll * 20
    pianoRoll = convertProbabilitiesToActualPianoRoll(decoded_pianoRoll)
    
    #converting probabilities 0-1 to velocity values 0-100
    pianoRoll = pianoRoll*100
    
    #print(pianoRoll.dtype)
    #pianoRoll = pianoRoll.astype('int32')
    #print(pianoRollInt.dtype)
    #Converting velocity values to values you can actually hear (from 1.5 to something like 60)
    #decoded_pianoRoll = decoded_pianoRoll*10
    
    
    #Inspecting values generated by the model
    #for i in range(200):
    #    print(pianoRoll[i])
    
    #Reshaping to become an actual pianoRoll
    pianoRoll = reshape(pianoRoll)
    print(pianoRoll.shape)
    
    #Flipping nodes around
    #pianoRoll = np.flip(pianoRoll, 0)
    
    #saving to midi
    pm = piano_roll_to_pretty_midi(pianoRoll, 50, 0)
    pm.write(pathToSave + "\\LSTM" + str(seq_index) + ".mid")
    
    #Adding pianoroll to list of all pianorolls
    #decodedPianoRolls = np.append(decodedPianoRolls, decoded_pianoRoll, axis=0)
    #pianoRolls = np.append(pianoRolls, pianoRoll, axis=0)

----------------
(1, 1500, 128)
(1501, 1, 128)
(1501, 128)
(128, 1500)
----------------
(1, 1500, 128)
(1501, 1, 128)
(1501, 128)
(128, 1500)
----------------
(1, 1500, 128)
(1501, 1, 128)
(1501, 128)
(128, 1500)


In [68]:
for i in range(200):
    print(decoded_pianoRoll[i])

[0.49288458 0.49135816 0.49544048 0.50504529 0.49697128 0.50487006
 0.49354589 0.51059628 0.49784964 0.49352273 0.49537855 0.49765036
 0.50865978 0.50240844 0.50400323 0.4909246  0.50357187 0.49313349
 0.51349485 0.50934035 0.49151742 0.50252849 0.51490539 0.51166683
 0.50395817 0.49172422 0.498445   0.50097764 0.50480437 0.49456394
 0.50701827 0.49507391 0.50250012 0.50722414 0.49526066 0.50285929
 0.49638799 0.48952603 0.50937498 0.49000484 0.48678148 0.50014979
 0.50091803 0.50370806 0.49068624 0.49477962 0.50300229 0.50110745
 0.49513835 0.49615061 0.50002235 0.50930589 0.49855039 0.49555534
 0.50082636 0.50046349 0.49778053 0.50298786 0.50014377 0.50565553
 0.50117266 0.4992899  0.50422907 0.49170128 0.50010216 0.50814682
 0.50800329 0.49714687 0.50712091 0.50510567 0.49782664 0.50544697
 0.49970305 0.48537242 0.49957326 0.50503254 0.51360863 0.5047614
 0.50717121 0.49530897 0.48712346 0.48860765 0.49958721 0.5073204
 0.49254075 0.50364161 0.49980783 0.49567544 0.50609893 0.499893

In [61]:
print(pianoRoll.shape)

(128, 1500)


In [42]:

print(pianoRoll[:,0])
print(reshape(pianoRollInt)[:,0])

[  0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.          20.59948444   0.           0.           0.
   0.           0.         676.67424679   0.           0.
   0.           0.           0.          10.23352146   0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.         

In [69]:
for i in range(200):
        print(pianoRoll[:,i])
        #print(flipped[:,i])

[ 0.          0.          0.          0.          0.          0.
  0.         51.05962753  0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 51.34948492  0.          0.          0.         51.49053931 51.16668344
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         51.36086345  0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0. 

In [60]:
print(pianoRoll[:,1499])

[ 0.          0.          0.          0.          0.         53.1111896
 51.95047855  0.          0.          0.          0.         55.96855283
 56.56837821  0.          0.          0.         54.44514751  0.
  0.          0.         55.15329242 59.36402082 55.27530909 57.27596283
 54.48746085 52.14750171 53.32520604 54.14920449  0.          0.
 56.69170022  0.          0.          0.          0.          0.
 54.9484849  56.26514554 53.07244658  0.          0.          0.
  0.         58.7882936   0.          0.         56.49045706  0.
  0.          0.         64.56117034  0.          0.         57.15601444
  0.          0.          0.         55.30728102 56.72240853  0.
 54.11682129  0.         58.71154666  0.         55.44787049  0.
  0.         51.95290446 51.74952149 57.30776787 52.0576179  58.31773877
  0.          0.         52.70258784  0.         53.75720263 51.45196915
  0.          0.         51.33750439 53.41284275  0.          0.
  0.         53.27670574 52.03086138  0.   

In [56]:
v_changes = np.nonzero(np.diff(pianoRoll).T)

In [57]:
print(np.array(v_changes).shape)
print(np.array(v_changes)[0][6])

(2, 85350)
0
